In [35]:
from IPython.display import clear_output
from nnsight import LanguageModel
from typing import List, Callable
import torch
import numpy as np
from IPython.display import clear_output

# Load gpt2
model = LanguageModel("openai-community/gpt2", device_map="auto", dispatch=True)


prompt= "The Eiffel Tower is in the city of"
layers = model.transformer.h
probs_layers = []

with model.trace() as tracer:
    with tracer.invoke(prompt) as invoker:
        for layer_idx, layer in enumerate(layers):
            # Process layer output through the model's head and layer normalization
            layer_output = model.lm_head(model.transformer.ln_f(layer.output[0]))

            # Apply softmax to obtain probabilities and save the result
            probs = torch.nn.functional.softmax(layer_output, dim=-1).save()
            probs_layers.append(probs)

probs = torch.cat([probs.value for probs in probs_layers])

# Find the maximum probability and corresponding tokens for each position
max_probs, tokens = probs.max(dim=-1)

# Decode token IDs to words for each layer
words = [[model.tokenizer.decode(t.cpu()).encode("unicode_escape").decode() for t in layer_tokens]
    for layer_tokens in tokens]

# Access the 'input_ids' attribute of the invoker object to get the input words
input_words = [model.tokenizer.decode(t) for t in invoker.inputs[0]["input_ids"][0]]



/home/allanz/miniconda3/envs/vlm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [41]:
print(words)
print(input_words)

print(len(words[0]))
print(len(input_words[0]))

[['\\n', ' E', 'iff', 'el', ' Tower', ' not', ' the', ' same', 'scape', ' the'], [' the', 'Z', 'ness', 'ter', ' Tower', ' not', ' order', ' same', 'scape', ' the'], [' the', 'leg', 'ness', 'ter', ' Tower', ' now', ' order', ' same', 'scape', ' the'], [' the', 'leg', 'ness', 'stra', ' Tower', ' also', ' the', ' same', ' hall', ' the'], [' the', 'leg', 'ness', 'horn', ' Tower', ' also', ' front', ' same', 'scape', ' the'], [' the', 'FP', 'ler', ' Tower', ' Towers', ' also', ' the', ' same', 'scape', ' the'], [' the', 'FP', 'ler', ' Tower', ' Tower', ' also', ' front', ' midst', 'scape', ' East'], [' the', 'FP', 'ler', ' Tower', ' Tower', ' located', ' front', ' midst', ' centre', ' Ing'], [' the', 'TS', 'ler', ' Tower', ' Tower', ' located', ' front', ' midst', ' centre', ' Rome'], [' the', 'FP', 'el', ' Tower', ' tower', ' located', ' London', ' midst', ' centre', ' London'], [' the', 'nerg', 'el', ' Tower', ' is', ' located', ' the', ' midst', "'s", ' Paris'], ['\\n', '-', 'el', ' Towe

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected+colab+notebook"

fig = px.imshow(
    max_probs.detach().cpu().numpy(),
    x=input_words,
    y=list(range(len(words))),
    color_continuous_scale=px.colors.diverging.RdYlBu_r,
    color_continuous_midpoint=0.50,
    text_auto=True,
    labels=dict(x="Input Tokens", y="Layers", color="Probability")
)

fig.update_layout(
    title='Logit Lens Visualization',
    xaxis_tickangle=0
)

fig.update_traces(text=words, texttemplate="%{text}")
fig.show()